In [ ]:
from run_honest_prover import honest_distance_error
import matplotlib.pyplot as plt
from run_attack import random_guess

# Honest prover

## 1. Honest prover against SPAM error, measurement error and photon loss

In [ ]:
p_err_no_error, distance, time = honest_distance_error(round=100, distance=50, x=1, y=0, z=3, measurement_error=0, loss_rate=0)
p_err_loss_only, distance, time = honest_distance_error(round=100, distance=50, x=1, y=0, z=3, measurement_error=0, loss_rate=0.2)
p_err_q_only, distance, time = honest_distance_error(round=100, distance=50, x=1, y=0, z=3, measurement_error=0.2, loss_rate=0)
p_err, distance, time = honest_distance_error(round=100, distance=50, x=1, y=0, z=3, measurement_error=0.2, loss_rate=0.2)

plt.figure(dpi=300)
plt.plot(distance, p_err, label='With all constraints', color='#C25759')
plt.plot(distance, p_err_loss_only, label='Photon loss only', color='#599CB4')
plt.plot(distance, p_err_no_error, label='Without any constraints', color='green')
plt.legend(loc='best')
plt.xlabel('Distance')
plt.ylabel('Error rate')
plt.ylim(-0.1, 1)

# Run malicious prover

## Random guess

In [ ]:
p_err, distance, time = random_guess(round=100, distance=50, x=1, y=0, z=3, measurement_error=0.2, loss_rate=0)
p_err_h, distance_h, time_h = honest_distance_error(round=100, distance=50, x=1, y=0, z=3, measurement_error=0.2, loss_rate=0)

fig, ax = plt.subplots(nrows=1, ncols=1, dpi=300)
ax.set_ylim(0, 1)
ax.plot(distance, p_err, label='Random Guess')
ax.plot(distance, p_err_h, label='Honest prover')
fig.patches.extend([plt.Rectangle((0.4,0.6),0.48,0.27,
                                  fill=True, color='g', alpha=0.2,
                                  transform=fig.transFigure, figure=fig)])
plt.legend(loc='best')
plt.show()

## Run the MonE Game